In [22]:
import nba_api
import pandas as pd
import sklearn
import os
import numpy as np
import requests
import time
from sklearn.model_selection import KFold
import datetime as dt
import seaborn as sns
from nba_api.stats.static import players
import matplotlib.pyplot as plt
import math
from datetime import datetime, timedelta
from nba_api.stats.library.parameters import SeasonAllNullable
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
from tqdm import tqdm
from scipy.stats import norm
from nba_api.stats import endpoints
from nba_api.stats.static import teams
from sklearn.ensemble import RandomForestClassifier
from nba_api.stats.library.parameters import SeasonType
from nba_api.stats.library.parameters import SeasonTypePlayoffs
from nba_api.stats.library.parameters import  SeasonNullable

In [23]:
### Get list of NBA teams

nba_team = pd.DataFrame(teams.get_teams())
nba_team

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


In [30]:
### Get all games from NBA seasons since 2016

allseasons = ['2000-01','2001-02','2002-03','2003-04',
              '2004-05','2005-06','2006-07','2007-08','2008-09','2009-10', 
              '2010-11', '2011-12','2012-13','2013-14','2014-15',
              '2015-16','2016-17','2017-18','2018-19','2019-20','2020-21']

games = pd.DataFrame()
for season in tqdm(allseasons):
    gameappend_regular = endpoints.leaguegamefinder.LeagueGameFinder(season_type_nullable = SeasonType.regular, 
                                              season_nullable = season).get_data_frames()[0]
    gameappend_playoff = endpoints.leaguegamefinder.LeagueGameFinder(season_type_nullable = SeasonTypePlayoffs.playoffs, 
                                              season_nullable = season).get_data_frames()[0]
    gameappend_regular['PLAYOFF'] = 0 
    gameappend_playoff['PLAYOFF'] = 1
    games = games.append(pd.DataFrame(gameappend_regular))
    games = games.append(pd.DataFrame(gameappend_playoff))
    time.sleep(1)
    

games['WL'] = games['WL'].eq('W').mul(1)
NBA_codes = nba_team['id']
NBAgames = games[games['TEAM_ID'].isin(np.array(NBA_codes))]

## ALSO: Order by time, format times correctly

NBAgames = NBAgames.sort_values(by=['GAME_DATE'], ascending = 0).reset_index(drop = 1)
NBAgames['GAME_DATE'] = pd.to_datetime(NBAgames['GAME_DATE'])

NBAgames['TEAM_ID'] = NBAgames['TEAM_ID'].astype(str)
NBAgames = NBAgames[NBAgames['GAME_DATE'] != datetime.today().strftime('%Y-%m-%d')]

### Now need to combine the same games listed twice for the home and away team
## Must filter to unique games by combinding home and away games (vs or @ in mathchup)

NBAgamesC = pd.DataFrame()
for game_id in tqdm(np.unique(NBAgames['GAME_ID'])):
    game_match = pd.DataFrame(NBAgames[NBAgames['GAME_ID'] == game_id])
    home_game = pd.DataFrame(game_match[game_match['MATCHUP'].str.contains('vs')])
    away_game = pd.DataFrame(game_match[game_match['MATCHUP'].str.contains('@')]).drop(['SEASON_ID',
                                                                                        'GAME_DATE',
                                                                                        'GAME_ID'
                                                                                       ], axis =1)
    home_game.columns = [str(col) + '_H' for col in home_game.columns]
    away_game.columns = [str(col) + '_A' for col in away_game.columns]
    combined = pd.DataFrame(pd.concat([home_game.reset_index(drop=True), away_game.reset_index(drop=True)], axis = 1))
    NBAgamesC = pd.concat([NBAgamesC, combined])

    
NBAgamesC['GAME_DATE_H'] = pd.to_datetime(NBAgamesC['GAME_DATE_H'])

# Find Team swithes and change to new team name

NBAgamesC = NBAgamesC.drop(['PLUS_MINUS_A','PLUS_MINUS_H'], axis = 1).replace({'NOH': 'NOP',
                              'NJN': 'BKN',
                              'SEA': 'OKC',
                              'VAN': 'MEM',
                              'KCK': 'SAC',
                              'CHH': 'CHA',
                              'SDC': 'LAC',
                              'UTH': 'UTA',
                              'SAN': 'SAS',
                              'GOS': 'GSW',
                              'PHL': 'PHI',
                              'NOH': 'NOP',
                              'New Orleans Hornets': 'New Orleans Pelicans',
                              'NOK': 'NOP',
                             'New Orleans/Oklahoma City Hornets': 'New Orleans Pelicans',
                              'New Orleans Hornets': 'New Orleans Pelicans',
                              'New Jersey Nets': 'Brooklyn Nets',
                              'Seattle SuperSonics': 'Oklahoma City Thunder',
                              'Vancouver Grizzlies': 'Memphis Grizzlies',
                              'Kansas City Kings': 'Sacramento Kings',
                              'San Diego Clippers': 'Los Angeles Clippers'}, regex=True).sort_values(by=['GAME_DATE_H'], 
                                                                                                     ascending = 0).reset_index(drop = 1)
NBAgamesC['SEASON_ID_H'] = NBAgamesC['SEASON_ID_H'].str[1:]

100%|██████████| 27814/27814 [09:07<00:00, 50.80it/s]


In [31]:
# Find games with a team that is not in the NBA and remove

nonNBA = []
for index in tqdm(range(len(np.unique(NBAgamesC['MATCHUP_H'].astype(str))))):
    teams = np.unique(NBAgamesC['MATCHUP_H'].astype(str))[index]
    t1 = teams[0:3]
    t2 = teams[-3:]
    if not t1 in np.array(nba_team['abbreviation']): 
        nonNBA = np.append(nonNBA, t1)
        continue
    if not t2 in np.array(nba_team['abbreviation']):
        nonNBA = np.append(nonNBA, t2)
        
remove = pd.DataFrame()
for team in nonNBA:
    removeappend = NBAgamesC[NBAgamesC['MATCHUP_H'].str.contains(team).fillna(False)]
    removeappend2 = NBAgamesC[NBAgamesC['MATCHUP_A'].str.contains(team).fillna(False)]
    remove = remove.append(removeappend)
    remove = remove.append(removeappend2)
    
NBAgamesC = pd.merge(NBAgamesC,remove, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)

## Remove low amounts of missing data (ok with large sample size), create win margin feature
NBAgamesC = NBAgamesC[~NBAgamesC.isna().any(axis=1)] # remove rows with NA (VERY FEW) shouldnt cause bias
NBAgamesC['WINMARGIN_H'] = NBAgamesC['PTS_H'] - NBAgamesC['PTS_A']
NBAgamesC = NBAgamesC.reset_index(drop=1)

NBAgamesC['TEAM_ID_H'] = NBAgamesC['TEAM_ID_H'].astype(int)
NBAgamesC['TEAM_ID_A'] = NBAgamesC['TEAM_ID_A'].astype(int)

NBAgamesC.drop(columns = 'PLAYOFF_A')

NBAgamesC ## FINAL DATA

100%|██████████| 898/898 [00:18<00:00, 49.08it/s]


,SEASON_ID_H,TEAM_ID_H,TEAM_ABBREVIATION_H,TEAM_NAME_H,GAME_ID_H,GAME_DATE_H,MATCHUP_H,WL_H,MIN_H,PTS_H,FGM_H,FGA_H,FG_PCT_H,FG3M_H,FG3A_H,FG3_PCT_H,FTM_H,FTA_H,FT_PCT_H,OREB_H,DREB_H,REB_H,AST_H,STL_H,BLK_H,TOV_H,PF_H,PLAYOFF_H,TEAM_ID_A,TEAM_ABBREVIATION_A,TEAM_NAME_A,MATCHUP_A,WL_A,MIN_A,PTS_A,FGM_A,FGA_A,FG_PCT_A,FG3M_A,FG3A_A,FG3_PCT_A,FTM_A,FTA_A,FT_PCT_A,OREB_A,DREB_A,REB_A,AST_A,STL_A,BLK_A,TOV_A,PF_A,PLAYOFF_A,WINMARGIN_H
0,2020,1610612749,MIL,Milwaukee Bucks,0042000406,2021-07-20,MIL vs. PHX,1.0,240.0,105.0,37.0,82.0,0.451,6.0,27.0,0.222,25.0,29.0,0.862,11.0,42.0,53.0,20.0,10.0,6.0,18.0,17.0,1.0,1610612756,PHX,Phoenix Suns,PHX @ MIL,0.0,239.0,98.0,38.0,86.0,0.442,6.0,25.0,0.240,16.0,19.0,0.842,6.0,31.0,37.0,14.0,11.0,4.0,14.0,21.0,1.0,7.0
1,2020,1610612756,PHX,Phoenix Suns,0042000405,2021-07-17,PHX vs. MIL,0.0,241.0,119.0,48.0,87.0,0.552,13.0,19.0,0.684,10.0,11.0,0.909,8.0,27.0,35.0,23.0,9.0,5.0,8.0,20.0,1.0,1610612749,MIL,Milwaukee Bucks,MIL @ PHX,1.0,240.0,123.0,50.0,87.0,0.575,14.0,28.0,0.500,9.0,17.0,0.529,11.0,26.0,37.0,26.0,7.0,1.0,11.0,17.0,1.0,-4.0
2,2020,1610612749,MIL,Milwaukee Bucks,0042000404,2021-07-14,MIL vs. PHX,1.0,240.0,109.0,39.0,97.0,0.402,7.0,29.0,0.241,24.0,29.0,0.828,17.0,31.0,48.0,22.0,11.0,3.0,5.0,18.0,1.0,1610612756,PHX,Phoenix Suns,PHX @ MIL,0.0,240.0,103.0,40.0,78.0,0.513,7.0,23.0,0.304,16.0,19.0,0.842,5.0,35.0,40.0,18.0,3.0,9.0,17.0,24.0,1.0,6.0
3,2020,1610612749,MIL,Milwaukee Bucks,0042000403,2021-07-11,MIL vs. PHX,1.0,240.0,120.0,43.0,90.0,0.478,14.0,36.0,0.389,20.0,26.0,0.769,13.0,34.0,47.0,28.0,8.0,2.0,9.0,18.0,1.0,1610612756,PHX,Phoenix Suns,PHX @ MIL,0.0,239.0,100.0,40.0,83.0,0.482,9.0,31.0,0.290,11.0,16.0,0.688,6.0,30.0,36.0,21.0,3.0,3.0,14.0,24.0,1.0,20.0
4,2020,1610612756,PHX,Phoenix Suns,0042000402,2021-07-08,PHX vs. MIL,1.0,240.0,118.0,43.0,88.0,0.489,20.0,40.0,0.500,12.0,14.0,0.857,11.0,32.0,43.0,26.0,7.0,2.0,12.0,19.0,1.0,1610612749,MIL,Milwaukee Bucks,MIL @ PHX,0.0,240.0,108.0,42.0,93.0,0.452,9.0,31.0,0.290,15.0,23.0,0.652,18.0,28.0,46.0,21.0,8.0,7.0,9.0,17.0,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27734,2000,1610612762,UTA,Utah Jazz,0020000010,2000-10-31,UTA vs. LAC,1.0,239.0,107.0,43.0,85.0,0.506,5.0,10.0,0.500,16.0,31.0,0.516,11.0,22.0,33.0,33.0,11.0,4.0,12.0,30.0,0.0,1610612746,LAC,Los Angeles Clippers,LAC @ UTA,0.0,240.0,94.0,34.0,69.0,0.493,4.0,13.0,0.308,22.0,34.0,0.647,12.0,32.0,44.0,18.0,3.0,6.0,24.0,32.0,0.0,13.0
27735,2000,1610612744,GSW,Golden State Warriors,0020000011,2000-10-31,GSW vs. PHX,1.0,239.0,96.0,32.0,79.0,0.405,4.0,8.0,0.500,28.0,38.0,0.737,14.0,41.0,55.0,18.0,11.0,5.0,21.0,22.0,0.0,1610612756,PHX,Phoenix Suns,PHX @ GSW,0.0,240.0,94.0,36.0,91.0,0.396,6.0,21.0,0.286,16.0,20.0,0.800,11.0,33.0,44.0,25.0,12.0,3.0,16.0,28.0,0.0,2.0
27736,2000,1610612757,POR,Portland Trail Blazers,0020000012,2000-10-31,POR vs. LAL,0.0,239.0,86.0,34.0,85.0,0.400,4.0,11.0,0.364,14.0,16.0,0.875,13.0,19.0,32.0,18.0,13.0,1.0,10.0,28.0,0.0,1610612747,LAL,Los Angeles Lakers,LAL @ POR,1.0,239.0,96.0,36.0,63.0,0.571,5.0,11.0,0.455,19.0,30.0,0.633,8.0,31.0,39.0,28.0,5.0,8.0,20.0,18.0,0.0,-10.0
27737,2000,1610612751,BKN,Brooklyn Nets,0020000002,2000-10-31,BKN vs. CLE,0.0,239.0,82.0,31.0,85.0,0.365,3.0,10.0,0.300,17.0,26.0,0.654,12.0,35.0,47.0,24.0,9.0,8.0,12.0,31.0,0.0,1610612739,CLE,Cleveland Cavaliers,CLE @ BKN,1.0,240.0,86.0,32.0,78.0,0.410,2.0,7.0,0.286,20.0,32.0,0.625,11.0,41.0,52.0,16.0,5.0,8.0,19.0,27.0,0.0,-4.0


In [32]:
pd.DataFrame.to_csv(NBAgamesC, '/Users/nickbachelder/Buckets Model/NBAgamesC')
pd.DataFrame.to_csv(nba_team, '/Users/nickbachelder/Buckets Model/nba_team')
pd.DataFrame.to_csv(NBAgames, '/Users/nickbachelder/Buckets Model/NBAgames')